## LU FACTORIZATION FROM SCRATCH


In [1]:
import numpy as np

## Objective 

Decompose a given square matrix ${\bf A}$ in a product of a lower triangular matrix ${\bf L}$ and an upper triangular matrix ${\bf U}$:

$${\bf A} = {\bf L} {\bf U}$$


$$\begin{bmatrix}a_{11} & a_{12} & a_{13}\\
                 a_{21} & a_{22} & a_{23} \\
                 a_{31} & a_{32} & a_{33}
                \end{bmatrix} = \begin{bmatrix}l_{11} &  &  & {\Huge 0}\\
                           l_{21} & l_{22} &  & \\
                          \vdots & \vdots & \ddots & \\
                           l_{n1} & l_{n2} & \dots & l_{nn}
                \end{bmatrix} \cdot \begin{bmatrix} u_{11} & u_{12} & \dots & u_{1n}\\
                                                    & u_{22} & \dots & u_{2n}\\
                                                     &  & \ddots & \vdots\\
                                                    {\Huge 0} &  &  & u_{nn}
                                                    \end{bmatrix}$$

In [2]:
A = np.array([[2,3,1],[1,2,3],[3,1,2]])
A

array([[2, 3, 1],
       [1, 2, 3],
       [3, 1, 2]])

## 1. Obtain ${\bf U}$

Obtain ${\bf U}$ from ${\bf A}$ by applying the following row operations (Singh,2013):

*  Multiplying a row by a non-zero constant.
* Adding a multiple of one row to another

(However, I noticed that when your objective is to solve a system ${\bf A}{\bf x}={\bf b}$, only the second operation is essential, you can arrive to the same result by ignoring the first row operation)

The following entries need to be zero in our matrix ${\bf A}$:

$$\begin{bmatrix}a_{11} & a_{12} & a_{13}\\
                 \color{orange}{a_{21}} & a_{22} & a_{23} \\
                 \color{orange}{a_{31}} & \color{orange}{a_{32}} & a_{33}
                \end{bmatrix}$$
                
We can proceed from left to right (column 1 and then column 2) and from top to bottom (row 1 and then row 2): $a_{21},a_{31},a_{32}$. That sequence can be easily made with two ```for``` loops.

I observed that a possible operation to convert a given entry  below the main diagonal $a_{ij}$ to zero is: $a_{ij} - \frac{a_{ij}}{a_{jj}}a_{jj}$. The ith row ${\bf r}_i$, which contains that entry, is altered like this: ${\bf r}_i^* = {\bf r}_i - \frac{a_{ij}}{a_{jj}}{\bf r}_{j}$. That means that the procedure affects the matrix this way:

$$\begin{bmatrix}1 & 2 & 2\\
                 3 & -3 & -2 \\
                 4 & -1 & -5
                \end{bmatrix} \rightarrow
                \begin{bmatrix}a_{11} & a_{12} & a_{13}\\
                 a_{21}-\frac{a_{21}}{a_{11}}a_{11} & a_{22}-\frac{a_{21}}{a_{11}}a_{12} & a_{32}-\frac{a_{21}}{a_{11}}a_{13} \\
                 a_{31} & a_{32} & a_{33}
                \end{bmatrix} = \begin{bmatrix}1 & 2 & 2\\
                 3-\frac{3}{1}\cdot 1 & -3-\frac{3}{1}\cdot 2 & -2-\frac{3}{1}\cdot 2 \\
                 4 & -1 & -5
                \end{bmatrix} = \begin{bmatrix}1 & 2 & 2\\
                 0 & -9 & -8 \\
                 4 & -1 & -5
                \end{bmatrix}$$
                
The process is repeated with each entry.

Note that we avoid "losing" the zeros already created. Once we finish with the first column (entries $a_{21}$ and $a_{31}$) and turn to the second column (entry $a_{32}$) we no longer use the first row to make the zero. That would mean going back because we would risk losing the zero from $a_{31}$. 

$$\begin{bmatrix}a_{11} & a_{12} & a_{13}\\
                 0 & a_{22}^* & a_{32}^* \\
                 0 & a_{32} & a_{33}
                \end{bmatrix}$$

Instead, we use now the second row, which contains a zero in $a_{21}$, so it can't affect $a_{31}$.



## 2. Obtain ${\bf L}$

The ${\bf L}$ matrix is obtained from the identity ${\bf I}$ and the process is very straightforward once you have found ${\bf U}$. You just need to replace this zero entries in ${\bf I}$

$$\begin{bmatrix}1 & 0 & 0\\
                 \color{orange}{i_{21}} & 1 & 0 \\
                 \color{orange}{i_{31}} & \color{orange}{i_{32}} & 1
                \end{bmatrix}$$
                
with the negative of the multipliers used to obtain ${\bf U}$. Those multipliers are just the scalars $\frac{a_{ij}}{a_{jj}}$. Continuing with the example from before, to convert $a_{21}$ to zero we executed the row operation ${\bf r}_2^* = {\bf r}_2 + (- 3)\cdot {\bf r}_1$, so the negative multiplier is 3 and the modified identity matrix is now

$$\begin{bmatrix}1 & 0 & 0\\
                 3 & 1 & 0 \\
                 i_{31} & i_{32} & 1
                \end{bmatrix}$$
                
The process is repeated with each entry. 

In [3]:
def LU_decomp(A,n):
    startentry = 0
    U = A.copy()
    L = np.identity(n)
                    
    for col in range(n - 1): # (if A is n x n) from col 0 to col n-1
        startentry += 1 
        for row in range(startentry,n): # from row 1 to row n in the first col,
            multiplier = -U[row,col]/U[col,col]  # from row 2 to row n in the second col ...
            U[row] = U[row] + multiplier * U[col]
            L[row,col] = -multiplier
            
    return L,U

In [4]:
L,U = LU_decomp(A, n = A.shape[0])

print(L);print(U)

[[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 1.5 -inf  1. ]]
[[          2           3           1]
 [          0           0           2]
 [-2147483648 -2147483648 -2147483648]]


<ipython-input-3-bd780ef0d2b6>:9: RuntimeWarning: divide by zero encountered in long_scalars
  multiplier = -U[row,col]/U[col,col]  # from row 2 to row n in the second col ...
<ipython-input-3-bd780ef0d2b6>:10: RuntimeWarning: invalid value encountered in multiply
  U[row] = U[row] + multiplier * U[col]


In [5]:
# check if the decomposition is right: A = L * U

A == np.dot(L,U)

array([[ True,  True,  True],
       [ True, False, False],
       [False, False, False]])

## 3. Solve ${\bf A}{\bf x}={\bf b}$

The LU factorization can serve to solve a system ${\bf A}{\bf x}={\bf b}$. In order to do that we need to solve two different (easier) systems:

* ${\bf L}{\bf y}={\bf b}$
* ${\bf U}{\bf x}={\bf y}$

In [6]:
# let's suppose that
b = np.array([106.8,177.2,279.2])
b

array([106.8, 177.2, 279.2])

### 3.1 ${\bf L}{\bf y}={\bf b}$

$$\begin{bmatrix}l_{11} & 0 & 0\\
                 l_{21} & l_{22} & 0 \\
                 l_{31} & l_{32} & l_{33}
                \end{bmatrix} \cdot \begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix} = \begin{bmatrix} b_1 \\ b_2 \\ b_3 \end{bmatrix}$$
                
$$
\begin{aligned}
l_{11}y_1 & = b_1 \; ; \; y_1 = \frac{b_1}{l_{11}}\\
l_{21}y_1+l_{22}y_2 & = b_2 \; ; \; y_2 = \frac{b_2 -l_{21}y_1}{l_{22}}\\
l_{31}y_1+l_{32}y_2+l_{33}y_3 & = b_3 \; ; \; y_3 = \frac{b_3 -(l_{31}y_1+l_{32}y_2)}{l_{33}}
\end{aligned}$$

In our example:

$$\begin{bmatrix}1 & 0 & 0\\
                 3 & 1 & 0 \\
                 4 & 1 & 1
                \end{bmatrix} \cdot \begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix} = \begin{bmatrix} 5 \\ 0 \\ 10 \end{bmatrix}$$
                
$$
\begin{aligned}
y_1 & = 5 \; ; \; y_1 = 5\\
3y_1+y_2 & = 0 \; ; \; y_2 =-3y_1 = -15\\
4y_1+y_2+y_3 & = 10 \; ; \; y_3 =10-(4y_1+y_2) = -15
\end{aligned}$$

Despite the appearance, each equation has only one unkown, the others are obtained from the previous equations. In the general case, when ${\bf L}$ is $n \times n$:

$$y_i = \frac{b_i-\sum_j^n l_{ij} \cdot y_j}{l_{ii}}$$

In [7]:
y = np.zeros(L.shape[0])
Lcols = L.shape[1]

for i in range(len(y)):
    y[i] = (b[i] - np.sum([L[i,j]*y[j] for j in range(Lcols)]))/L[i,i]
y

array([106.8, 123.8,   inf])

### 3.2 ${\bf U}{\bf x}={\bf y}$

$$\begin{bmatrix} u_{11} & u_{12} & u_{13}\\
                  0 & u_{22} & u_{23}\\
                  0 & 0 & u_{33}\end{bmatrix}\cdot \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix}$$
                  
(Starting from the bottom)

$$
\begin{aligned}
u_{33}x_3 &= y_3 \; ; \; x_3 = \frac{y_3}{u_{33}}\\
u_{22}x_2+u_{23}x_3 &= y_2 \; ; \; x_2 = \frac{y_2-u_{23}x_3}{u_{22}}\\
u_{11}x_1+u_{12}x_2+u_{13}x_3 &= y_1 \; ; \; x_1 = \frac{y_1-(u_{12}x_2+u_{13}x_3)}{u_{11}}
\end{aligned}
$$

In our example:

$$\begin{bmatrix}1 & 2 & 2\\
                 0 & -9 & -8 \\
                 0 & 0 & -5
                \end{bmatrix} \cdot \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} 5 \\ -15 \\ -15 \end{bmatrix}$$
                
$$
\begin{aligned}
-5x_3 &= -15 \; ; \; x_3 = \frac{-15}{-5}=3\\
-9x_2-8x_3 &= -15 \; ; \; x_2 = \frac{-15+8x_3}{-9}=-1\\
x_1+2x_2+2x_3 &= 5 \; ; \; x_1 = 5-(2x_2+2x_3) = 1
\end{aligned}
$$

Again,each equation has only one unkown, the others are obtained from the previous equations. In the general case, when ${\bf U}$ is $n \times n$:

$$x_i = \frac{y_i-\sum_j^n u_{ij} \cdot x_j}{u_{ii}}$$

In [8]:
x = np.zeros(U.shape[0])
Ucols = U.shape[1]

for i in range(len(x)-1,-1,-1):
    x[i] = (y[i] - np.sum([U[i,j]*x[j] for j in range(Ucols)]))/U[i,i]
x

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


array([ nan,  inf, -inf])

In [9]:
# we can wrap everything into a function
def solve_with_LU(L,U,b,n):
    
    y = np.zeros(n)
    for i in range(len(y)):
        y[i] = (b[i] - np.sum([L[i,j]*y[j] for j in range(n)]))/L[i,i]
        
    x = np.zeros(n)
    for i in range(len(x)-1,-1,-1):
        x[i] = (y[i] - np.sum([U[i,j]*x[j] for j in range(n)]))/U[i,i]
        
    return x

In [10]:
my_solution = solve_with_LU(L,U,b, n = L.shape[0])
solution = np.linalg.solve(A,b)

my_solution == solution

array([False, False, False])

## Final function

In [11]:
def LU(A, b = None):
    """performs LU decomposition and/or solves the linear system Ax=b
    
    Parameters
    ----------
    A : 2-D n by n numpy array (square matrix)
        Square matrix to be decomposed.
    b : 1-D numpy array
        Optional vector.

    Returns
    -------
    tuple.
        Two element tuple containing the lower triangular matrix and
        the upper triangular matrix: (L,U).
        
    numpy array.
        If the only interest is to solve a given linear system Ax=b,
        the solution vector x is returned after applying the LU decomposition."""
    
    if A.shape[0] != A.shape[1]: return 'LU decomposition works with square matrices'
    
    n = A.shape[0]
    decomp_mats = LU_decomp(A,n)
    if b is None: return decomp_mats
    else: return solve_with_LU(decomp_mats[0],decomp_mats[1],b,n)

In [12]:
# some examples from Singh (2013)

B = np.array([[1,2,3,4],[17,22,27,8],[77,44,47,-494],[-10,1,7,63]])
c = np.array([-10,22,2106,-243])

print(LU(B,c))
print(np.linalg.solve(B,c))

[ 1. -2.  3. -4.]
[ 1. -2.  3. -4.]


In [13]:
B_not_square = B.reshape([2,8])
LU(B_not_square)

'LU decomposition works with square matrices'

In [14]:
C = np.array([[2,1,3],[4,-1,0],[10,12,34]])
d = np.array([5,-11,84])

L,U = LU(C)
C == np.dot(L,U)

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [15]:
np.random.seed(666)
D = np.random.randint(0,100,9).reshape([3,3])
e = np.random.randint(0,100,3)

print(LU(D,e))
print(np.linalg.solve(D,e))
print(np.linalg.lstsq(D,e)[0]) # least squares solution

[-0.82868227  2.02754311  0.04726415]
[-0.82842102  2.02792099  0.04667992]
[-0.82842102  2.02792099  0.04667992]


<ipython-input-15-1e53f450a38e>:7: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  print(np.linalg.lstsq(D,e)[0]) # least squares solution


When I create random matrices and vectors, the results are not exactly the same, but continue to be pretty similar


## References

* Singh, K. (2013). LU Factorization. In K. Singh, _Linear algebra: step by step_ (pp.472-481). Oxford University Press.